In [1]:
import pandas as pd

In [ ]:
# 讀取分K檔案
x = pd.read_csv('TXF.csv')
print(x)


                   Date   Open   High    Low  Close  Volume
0        2019/3/4 08:46  10390  10400  10380  10386    3858
1        2019/3/4 08:47  10387  10388  10373  10377    1332
2        2019/3/4 08:48  10377  10378  10371  10375     759
3        2019/3/4 08:49  10375  10376  10373  10376     436
4        2019/3/4 08:50  10375  10376  10361  10363     999
...                 ...    ...    ...    ...    ...     ...
267845  2020/2/28 04:56  11169  11171  11168  11170      49
267846  2020/2/28 04:57  11171  11172  11168  11168      64
267847  2020/2/28 04:58  11169  11169  11166  11166     169
267848  2020/2/28 04:59  11167  11167  11165  11166     143
267849  2020/2/28 05:00  11166  11166  11162  11162     182

[267850 rows x 6 columns]


In [ ]:
# 為 x 建立 index，供轉換使用
x = x.set_index(pd.DatetimeIndex(x.Date))
print(x)

# set_idnex() 是 Pandas DataFrame 的一個方法，用於將 DataFrmae 中的
# 一個或多個線有欄位設定為新的索引 (Index)
# pd.DatatimeIndex() 是 Pandas 中的一個用於創建日期時間索引物件的函式

# x.Date:這會從原始 DataFrame x 中選取出名為 Date 的那一欄數據 (包含交易日期或時間的字串/物件)
# pd.DatetimeIndex: 這個韓式接收 x.Date 的數據，並將其轉換成一個特殊的 Pandas DatetimeIndex 物件。

                                Date   Open   High    Low  Close  Volume
Date                                                                    
2019-03-04 08:46:00   2019/3/4 08:46  10390  10400  10380  10386    3858
2019-03-04 08:47:00   2019/3/4 08:47  10387  10388  10373  10377    1332
2019-03-04 08:48:00   2019/3/4 08:48  10377  10378  10371  10375     759
2019-03-04 08:49:00   2019/3/4 08:49  10375  10376  10373  10376     436
2019-03-04 08:50:00   2019/3/4 08:50  10375  10376  10361  10363     999
...                              ...    ...    ...    ...    ...     ...
2020-02-28 04:56:00  2020/2/28 04:56  11169  11171  11168  11170      49
2020-02-28 04:57:00  2020/2/28 04:57  11171  11172  11168  11168      64
2020-02-28 04:58:00  2020/2/28 04:58  11169  11169  11166  11166     169
2020-02-28 04:59:00  2020/2/28 04:59  11167  11167  11165  11166     143
2020-02-28 05:00:00  2020/2/28 05:00  11166  11166  11162  11162     182

[267850 rows x 6 columns]


In [5]:
# 錯誤的示範-早盤不加參數的 agg
df30 = x.resample('30min').agg({
    'Open':'first','High':'max','Low':'min', 'Close':'last', 'Volume':'sum'
}).dropna()

# 儲存成新的csv檔案
df30.to_csv('TXF_30_Test_2.csv')

### 為什麼上面的是錯誤示範？
上面的操作本身在數學上是正確的，但對於台指期 (TXF) 這類非24小時連續交易的商品來說，他會產生錯誤的K棒

台指期的交易時間通常是：
* 早盤（一班交易時段）：08:45~13:45
* 夜盤（盤後交易時段）：15:00~隔日05:00

如果原始數據x包含夜盤數據，當執行resample('30min')時，Pandas的預期行為是：
1. 無條件填補：它會從第一筆數據到最後一筆數據之間，每隔30分鐘就嘗試建立一個K棒
2. 跨時段合併：在交易結束（例如13:45）到夜盤開始（15:00）之間，有一個75分鐘的空擋。
3. 錯誤產生：如果一個 30 分鐘的週期橫跨了早盤收盤時間與夜盤開盤時間（例如 $13:30 \rightarrow 14:00$），這個 K 棒中可能會包含 13:30~13:45 的真實數據，但 13:45~14:00 之間沒有交易數據，此時 Pandas 可能會將下一個交易時段（夜盤）的數據錯誤地合併進來，或產生非法的 K 棒。在早盤和夜盤之間，以及在夜盤收盤到隔日早盤開盤之間，會產生大量的空白（NaN）K 棒。雖然您用了 .dropna() 移除這些完全空白的 K 棒，但對於跨時段的 K 棒（例如隔夜 K 棒），處理方式仍然是錯誤的。

In [ ]:
# 正確的方法

morning_date = x.between_time('08:45', '13:45')
afternoon_time = x.between_time('15:00', '05:00')

# 早上8:45 ~ 13:45 的資料做 resample，並使用 offset
morning_resampled = morning_date.resample(
    '30min', closed='right', label='right', offset='15min').agg({
        'Open':'first',
        'High':'max',
        'Low':'min',
        'Close':'last',
        'Volume':'sum'
    }).dropna()
print(morning_resampled)

# 下午15:00 ~ 隔日凌晨 3:00 的資料做 resample，不使用 offset
afternoon_resampled = morning_date.resample(
    '30min', closed='right', label='right', offset='15min'
).agg({
    'Open':'first',
    'High':'max',
    'Low':'min',
    'Close':'last',
    'Volume':'sum'
}).dropna()
print(afternoon_resampled)

                        Open     High      Low    Close  Volume
Date                                                           
2019-03-04 09:15:00  10390.0  10400.0  10308.0  10309.0   28947
2019-03-04 09:45:00  10309.0  10343.0  10294.0  10315.0   27199
2019-03-04 10:15:00  10314.0  10325.0  10301.0  10313.0   11852
2019-03-04 10:45:00  10313.0  10324.0  10276.0  10280.0   15081
2019-03-04 11:15:00  10279.0  10293.0  10279.0  10292.0    8394
...                      ...      ...      ...      ...     ...
2020-02-27 11:45:00  11325.0  11344.0  11315.0  11329.0    8389
2020-02-27 12:15:00  11329.0  11329.0  11282.0  11283.0   11232
2020-02-27 12:45:00  11282.0  11295.0  11246.0  11260.0   15699
2020-02-27 13:15:00  11258.0  11275.0  11237.0  11260.0   12341
2020-02-27 13:45:00  11260.0  11288.0  11256.0  11273.0   15402

[2420 rows x 5 columns]
                        Open     High      Low    Close  Volume
Date                                                           
2019-03-04 09:1

In [ ]:
# 合併早上和下午的 resample 結果
df30 = pd.concat([morning_resampled, afternoon_resampled]).sort_index()
df30.to_csv('TXF_30.csv')

In [8]:
type(morning_resampled)

pandas.core.frame.DataFrame